# Gradio interface

### Interface for individual data

In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

# Load models
lr_model = joblib.load('lr_model.pkl')
rf_model = joblib.load('rf_model.pkl')
xgb_model = joblib.load('xgb_model.pkl')
ann_model = load_model('ann_model.h5')

# Load scaler for feature scaling
scaler = joblib.load('scaler.pkl')  

# Function to preprocess and predict
def predict_asd(Q1, Q2, Q3, Q4, Q5, Q6, Q7, Q8, Q9, Q10, Age_in_Months, Qchat_Score, Gender, Jaundice_History, model_choice):
    # Prepare the input data
    sample = pd.DataFrame({
        'Q1': [Q1],
        'Q2': [Q2],
        'Q3': [Q3],
        'Q4': [Q4],
        'Q5': [Q5],
        'Q6': [Q6],
        'Q7': [Q7],
        'Q8': [Q8],
        'Q9': [Q9],
        'Q10': [Q10],
        'Age_in_Months': [Age_in_Months],
        'Qchat_Score': [Qchat_Score],
        'Gender': [Gender],  # Label Encoded value
        'Jaundice_History': [Jaundice_History],  # Label Encoded value
    })

    # Scale the sample for ANN
    sample_scaled = scaler.transform(sample)

    # Model prediction
    if model_choice == 'Logistic Regression':
        prediction = lr_model.predict(sample)[0]
        proba = lr_model.predict_proba(sample)[0][1]
    elif model_choice == 'Random Forest':
        prediction = rf_model.predict(sample)[0]
        proba = rf_model.predict_proba(sample)[0][1]
    elif model_choice == 'XGBoost':
        prediction = xgb_model.predict(sample)[0]
        proba = xgb_model.predict_proba(sample)[0][1]
    elif model_choice == 'ANN':
        prediction = (ann_model.predict(sample_scaled) > 0.5).astype(int)[0][0]
        proba = ann_model.predict(sample_scaled)[0][0]
    
    # Convert the prediction to a readable format
    prediction_text = "Yes" if prediction == 1 else "No"
    
    return f"Prediction: {prediction_text}", f"Probability: {proba:.2f}"

# Create the Gradio interface with custom styling
iface = gr.Interface(
    fn=predict_asd,
    inputs=[
        gr.Dropdown(choices=[0, 1], value=0, label='Q1: I often notice small sounds when others do not. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q2: I usually concentrate more on the whole picture, rather than the small details. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q3: I find it easy to do more than one thing at once. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q4: If there is an interruption, I can switch back to what I was doing very quickly. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q5: I frequently find that I don’t know how to keep a conversation going. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q6: When I’m reading a story, I find it difficult to work out the characters’ intentions. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q7: I enjoy social occasions. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q8: I find it easy to work out what someone is thinking or feeling just by looking at their face. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q9: New situations make me anxious. (0: No, 1: Yes)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Q10: I enjoy meeting new people. (0: No, 1: Yes)'),
        gr.Number(label='Age in Months', value=36),  # Use Number instead of Textbox for numerical input
        gr.Number(label='Qchat Score', value=5),     # Use Number for numerical input
        gr.Dropdown(choices=[0, 1], value=1, label='Gender (0: Female, 1: Male)'),
        gr.Dropdown(choices=[0, 1], value=0, label='Jaundice History (0: No, 1: Yes)'),
        gr.Dropdown(choices=['Logistic Regression', 'Random Forest', 'XGBoost', 'ANN'], value='Logistic Regression', label='Select Model'),
    ],
    outputs=[
        gr.Textbox(label="Prediction", interactive=False, placeholder="Prediction will appear here..."),
        gr.Textbox(label="Probability", interactive=False, placeholder="Probability will appear here..."),
    ],
    live=False,  # Disable live update, we want the prediction only after button press
    title="Autism Spectrum Disorder (ASD) Prediction",  # Title for the page
    description="This interface predicts whether an individual may have Autism Spectrum Disorder (ASD) based on answers to various questions. Fill out the questionnaire and select a model to get predictions.",

)

# Launch the interface
iface.launch()


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.0 when using version 1.5.1. This might lead to breaking code or invalid resul

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Interface for entire dataset

In [2]:
import gradio as gr
import pandas as pd
import joblib
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load models and scaler
lr_model = joblib.load('lr_model.pkl')
rf_model = joblib.load('rf_model.pkl')
xgb_model = joblib.load('xgb_model.pkl')
ann_model = load_model('ann_model.h5')
scaler = joblib.load('scaler.pkl')  # Load the scaler used during training

# Function to process the uploaded dataset and return predictions
def process_and_predict(file):
    # Read the Excel file
    df = pd.read_excel(file.name)
    
    # Check if the required columns are present
    required_columns = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Age_in_Months', 'Qchat_Score', 'Gender', 'Jaundice_History']
    if not all(col in df.columns for col in required_columns):
        return "Error: The uploaded file does not contain all the required columns."
    
    # Prepare the data for prediction
    sample = df[required_columns]
    
    # Scale the data
    sample_scaled = scaler.transform(sample)

    # Make predictions using each model
    df['Prediction_LR'] = lr_model.predict(sample)
    df['Probability_LR'] = lr_model.predict_proba(sample)[:, 1]
    
    df['Prediction_RF'] = rf_model.predict(sample)
    df['Probability_RF'] = rf_model.predict_proba(sample)[:, 1]
    
    df['Prediction_XGB'] = xgb_model.predict(sample)
    df['Probability_XGB'] = xgb_model.predict_proba(sample)[:, 1]
    
    df['Prediction_ANN'] = (ann_model.predict(sample_scaled) > 0.5).astype(int).flatten()
    df['Probability_ANN'] = ann_model.predict(sample_scaled).flatten()
    
    # Save the result as a new Excel file
    output_file = "predictions_output.xlsx"
    df.to_excel(output_file, index=False)
    
    return output_file

# Gradio interface
iface = gr.Interface(
    fn=process_and_predict,
    inputs=gr.File(label="Upload Dataset (Excel)"),
    outputs=gr.File(label="Download Predicted Dataset"),
    title="ASD Prediction Based on Dataset",
    description="Upload an Excel dataset with the required columns, and the system will predict the ASD status using various models and return the updated dataset with predictions.",

)

# Launch the interface
iface.launch()


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersio

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Created dataset file at: .gradio\flagged\dataset4.csv


Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Created dataset file at: .gradio\flagged\dataset5.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
